In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from utils import *
from distance import *
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn_extra.cluster import KMedoids

In [9]:
meta, projects, votes = load_pb_ohe('data/poland_warszawa_2023_srodmiescie.pb')

In [20]:
def cluster(d: np.ndarray, k: int, **args):
    return KMedoids(n_clusters=k, metric='precomputed', **args).fit(d).labels_
def show_clusters(xs: np.ndarray, ids: np.ndarray, k: int):
    elems = [
        xs[ids == i]
        for i in range(k)
    ]
    for i, es in enumerate(elems):
        sums = es.sum(axis=0)
        size = es.shape[0]
        ps = np.argsort(sums)[::-1]
        print(f"Cluster {i+1}:")
        print(f"   Voters: {size/meta['num_votes']:.2%} ({size})")
        print(f"   Projects:")
        for id in ps[:8]:
            print(f"     {sums[id] / size : .2%} - [{id:2}] {projects.loc[id]['name']}")

In [11]:
d_jaccard = jaccard(votes, votes)
d_cosine = cosine(votes, votes)
d_chord = chord(votes, votes)

In [12]:
C = 0.9
d_simrank = simrank(votes, 0.9, 0.9, max_iter=1000)
d_simrank = (d_simrank - (1 - C)) / C
d_simrank[d_simrank < 0] = 0

Iteration 0 norm is (252.91391587217416, 0.015121093602125529)
Iteration 1 norm is (0.9878119741737995, 0.9751355504079687)
Iteration 2 norm is (73.04859816227768, 0.01111942708731678)
Iteration 3 norm is (0.7986134797472321, 0.7964697614659962)
Iteration 4 norm is (58.049385995321465, 0.008765107393857623)
Iteration 5 norm is (0.636361414084413, 0.6317840353634797)
Iteration 6 norm is (46.03245085269339, 0.006907851600687049)
Iteration 7 norm is (0.5034398917687158, 0.49833574272491993)
Iteration 8 norm is (36.35922548514709, 0.005442944257802915)
Iteration 9 norm is (0.39726924345208126, 0.39270208654713873)
Iteration 10 norm is (28.674267844144328, 0.004288367635962347)
Iteration 11 norm is (0.3131867832791055, 0.3094043326027651)
Iteration 12 norm is (22.599964980205442, 0.0033786129092780596)
Iteration 13 norm is (0.24680643476187517, 0.24376553737075152)
Iteration 14 norm is (17.808149656333313, 0.002661830820638493)
Iteration 15 norm is (0.19446550461260936, 0.1920497090608267)


In [13]:
k = 8

In [21]:
c_jaccard = cluster(d_jaccard, k)
c_cosine = cluster(d_cosine, k)
c_chord = cluster(d_chord, k)

In [25]:
c_simrank = cluster(d_simrank, k, method='alternate', init='k-medoids++')

ValueError: init needs to be one of the following: ['random', 'heuristic', 'k-medoids++', 'build', 'array-like']

In [15]:
show_clusters(votes, c_jaccard, 8)

Cluster 1:
   Voters: 37.21% (1445)
   Projects:
      37.23% - [ 9] Tęcza na bulwarach - wyświetlany hologram
      34.46% - [ 0] Rozbetonowanie i zazielenienie skrzyżowań na Muranowie
      32.73% - [ 5] Jadłodzielnia w Śródmieściu
      27.68% - [27] Ławka fotowoltaiczna z funkcją ładowania telefonów, tabletów i laptopów
      27.40% - [20] Spotkajmy się na Twardej w ogrodzie – otwarty ogród warzywny dla mieszkańców Warszawy i bezpłatne zajęcia
      26.85% - [ 2] Zieleń dla Powiśla – zagospodarowanie zieleni na skarpie warszawskiej
      26.71% - [ 4] Oczyszczenie stawu w Parku na Książęcem
      26.37% - [ 1] Zielona Wisłostrada
Cluster 2:
   Voters: 14.91% (579)
   Projects:
      56.65% - [ 8] Pomóż chronić stare cisy w Śródmieściu
      48.88% - [ 0] Rozbetonowanie i zazielenienie skrzyżowań na Muranowie
      48.36% - [ 1] Zielona Wisłostrada
      44.04% - [13] Zielony Zakątek na Łazienkowskiej
      43.18% - [ 5] Jadłodzielnia w Śródmieściu
      40.07% - [23] Mini park lino

In [16]:
show_clusters(votes, c_cosine, 8)

Cluster 1:
   Voters: 34.25% (1330)
   Projects:
      31.80% - [ 0] Rozbetonowanie i zazielenienie skrzyżowań na Muranowie
      31.35% - [18] Przebudowa placu zabaw przy Szkole Podstawowej nr 12 w Warszawie
      30.23% - [24] Modernizacja boiska szkolnego przy SP 12 im. Powstańców Śląskich w Warszawie
      25.04% - [ 9] Tęcza na bulwarach - wyświetlany hologram
      23.01% - [ 7] Zielona ulica Anielewicza
      22.93% - [ 5] Jadłodzielnia w Śródmieściu
      22.63% - [ 8] Pomóż chronić stare cisy w Śródmieściu
      22.63% - [ 3] Podlewanie 500+ drzew w Śródmieściu
Cluster 2:
   Voters: 22.77% (884)
   Projects:
      69.23% - [ 2] Zieleń dla Powiśla – zagospodarowanie zieleni na skarpie warszawskiej
      66.63% - [ 1] Zielona Wisłostrada
      59.16% - [ 4] Oczyszczenie stawu w Parku na Książęcem
      57.35% - [ 0] Rozbetonowanie i zazielenienie skrzyżowań na Muranowie
      56.67% - [13] Zielony Zakątek na Łazienkowskiej
      52.49% - [ 3] Podlewanie 500+ drzew w Śródmieściu


In [17]:
show_clusters(votes, c_chord, 8)

Cluster 1:
   Voters: 42.60% (1654)
   Projects:
      46.86% - [ 0] Rozbetonowanie i zazielenienie skrzyżowań na Muranowie
      46.31% - [ 1] Zielona Wisłostrada
      42.74% - [ 2] Zieleń dla Powiśla – zagospodarowanie zieleni na skarpie warszawskiej
      40.45% - [ 4] Oczyszczenie stawu w Parku na Książęcem
      37.30% - [ 9] Tęcza na bulwarach - wyświetlany hologram
      36.64% - [ 6] Zielone i doświetlone ulice, czyli przywracamy stylowe latarnie na ulicach Śródmieścia
      36.58% - [ 8] Pomóż chronić stare cisy w Śródmieściu
      36.34% - [ 3] Podlewanie 500+ drzew w Śródmieściu
Cluster 2:
   Voters: 15.09% (586)
   Projects:
      62.46% - [14] Zbiory dla bibliotek
      55.29% - [ 0] Rozbetonowanie i zazielenienie skrzyżowań na Muranowie
      48.81% - [ 3] Podlewanie 500+ drzew w Śródmieściu
      48.46% - [ 8] Pomóż chronić stare cisy w Śródmieściu
      47.78% - [17] Rewitalizacja drzew i mis na Polnej
      45.90% - [ 5] Jadłodzielnia w Śródmieściu
      45.39% - [ 1]

In [18]:
show_clusters(votes, c_simrank, 8)

Cluster 1:
   Voters: 100.00% (3883)
   Projects:
      43.01% - [ 0] Rozbetonowanie i zazielenienie skrzyżowań na Muranowie
      40.46% - [ 1] Zielona Wisłostrada
      37.37% - [ 2] Zieleń dla Powiśla – zagospodarowanie zieleni na skarpie warszawskiej
      35.00% - [ 3] Podlewanie 500+ drzew w Śródmieściu
      33.76% - [ 4] Oczyszczenie stawu w Parku na Książęcem
      33.69% - [ 5] Jadłodzielnia w Śródmieściu
      33.27% - [ 6] Zielone i doświetlone ulice, czyli przywracamy stylowe latarnie na ulicach Śródmieścia
      31.34% - [ 7] Zielona ulica Anielewicza
Cluster 2:
   Voters: 0.00% (0)
   Projects:
      nan% - [46] Preludium – wystawa i dyskusja na temat koncepcji rozwoju ścisłego centrum Warszawy
      nan% - [11] Stojaki rowerowe w Śródmieściu
      nan% - [20] Spotkajmy się na Twardej w ogrodzie – otwarty ogród warzywny dla mieszkańców Warszawy i bezpłatne zajęcia
      nan% - [19] Zielona stacja Gdańska
      nan% - [18] Przebudowa placu zabaw przy Szkole Podstawowej nr

/tmp/ipykernel_192504/2695044947.py:16: RuntimeWarning: invalid value encountered in scalar divide
  print(f"     {sums[id] / size : .2%} - [{id:2}] {projects.loc[id]['name']}")
